# Wrapper Approach - Hill Climbing
In this notebook we implement a rather simple feature selection procedure that follows a wrapper approach. The search algorithm, hill climbing in this case, is wrapped around the target classification/regression algorithm.

Sometimes we want to use some cheaper models. So we can use an approximated model that approximate the performance. This is beneficial in the computational cost even though the results are not as accurate as the 'original' (big) model.

If we have a dataset $D=\{X, t\}$, where $X = [x_1, x_2, \dots, x_M]$. It may be that only a subset of the features are relevant. So we may build a binary-valued vector that tells us which features have to be taken or not, e.g., $X_{sel} = [1, 0, \dots, 0]$. The fitness we will use, meanwhile, will be the cross-validation error using $X_{sel}$.

First we import the libraries that we will need.

In [5]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

from sklearn import datasets
from sklearn import linear_model
from sklearn.neighbors import KNeighborsRegressor

from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

Next we load the data and generate the k-fold evaluations.

In [6]:
data = datasets.load_boston()

scaler = StandardScaler()
X = scaler.fit_transform(data["data"])
y = data["target"]

number_of_variables = X.shape[1]
input_variables = data.feature_names
target_variable = 'MEDV'

seed = 1234
np.random.seed(seed)

# let's create also a pandas data frame
df = pd.DataFrame(data.data, columns=data.feature_names)
df['MEDV'] = y
df.head()

kfolds = KFold(10, shuffle=True, random_state=seed)

When applying a wrapper approach we are searching for the best subset of feature for a target model. In this case we will search for the best subset of features for plain linear regression.

In [10]:
def EvaluateFeatureSubsetSingleObjective(individual):
    selected_columns = []
    for i, allele in enumerate(individual):
        if (allele==1):
            selected_columns.append(df.columns[i])

    model = linear_model.LinearRegression()
    scores = cross_val_score(model, df[selected_columns], y, cv=kfolds)
    return scores.mean()

## Hill Climbing

In [15]:
def HillClimbing(number_of_variables, number_of_evaluations, evaluation_function):

    # current evaluation
    evaluations = 0
    
    # start from a random set of features
    current_feature_subset = [random.randint(0, 1) for x in range(number_of_variables)]

    # that will also provide an initial evaluation of the best performance
    best_performance = evaluation_function(current_feature_subset)
    
    print("%5d\t\t%3.2f\t%s"%(evaluations, best_performance, str(current_feature_subset)))
    
    # continue until all the evaluations have been performed
    # Most simple way for defining the stopping criteria.
    while evaluations < number_of_evaluations:
        
        # generate a neighbor candidate using a 10% perturbation of the current subset
        # BIT-FLIP
        # Probability would be better to be an hyperparameter to be learned (pass it to the function)
        perturbation = [(lambda x: 1-x if (random.random() < 0.1) else x)(x) for x in current_feature_subset]

        # evaluate only if there is at least one variable
        if (sum(perturbation)>0):
            performance = evaluation_function(perturbation)

            if (performance > best_performance):
                best_performance = performance
                current_feature_subset = perturbation

        evaluations = evaluations + 1
        print("%5d\t\t%3.2f\t%s"%(evaluations, best_performance, str(current_feature_subset)))

    print("Best Feature Subset = %s "%(str(current_feature_subset)))
    print("Performance = %3.2f"%(best_performance))

Let's run hill-climbing for 100 evaluations. 

In [17]:
HillClimbing(number_of_variables,400,EvaluateFeatureSubsetSingleObjective)

    0		0.51	[1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0]
    1		0.51	[1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0]
    2		0.51	[1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0]
    3		0.51	[1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0]
    4		0.64	[1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1]
    5		0.64	[1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1]
    6		0.64	[1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1]
    7		0.64	[1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1]
    8		0.64	[1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1]
    9		0.64	[1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1]
   10		0.64	[1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1]
   11		0.64	[1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1]
   12		0.64	[1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1]
   13		0.64	[1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1]
   14		0.65	[1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1]
   15		0.65	[1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1]
   16		0.65	[1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1]
   17		0.65	[1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1]
   18		0.65	[1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1]
   19		0.65	

We can repeat the same process targeting another model like for instance a k-nearest-neighbour regressor with a k of 5.

In [18]:
def EvaluateFeatureSubsetKNN(individual):
    selected_columns = []
    for i,allele in enumerate(individual):
        if (allele==1):
            selected_columns.append(df.columns[i])

    model = KNeighborsRegressor(5)
    scores = cross_val_score(model, df[selected_columns], y, cv=kfolds)
    return scores.mean()

In [19]:
HillClimbing(number_of_variables, 100, EvaluateFeatureSubsetKNN)

    0		0.23	[0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0]
    1		0.23	[0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0]
    2		0.31	[0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0]
    3		0.33	[1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0]
    4		0.34	[1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0]
    5		0.34	[1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0]
    6		0.34	[1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0]
    7		0.34	[1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0]
    8		0.35	[1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0]
    9		0.38	[1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0]
   10		0.39	[1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0]
   11		0.39	[1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0]
   12		0.39	[1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0]
   13		0.56	[1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1]
   14		0.56	[1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1]
   15		0.56	[1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1]
   16		0.56	[1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1]
   17		0.56	[1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1]
   18		0.56	[1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1]
   19		0.56	

## Discussion
Note that, with k-NN, we were able to reach a better performance with much fewer features. Might we draw some insight from this result? Also note that when doing feature selection we used the entire dataset but feature selection is, as a matter of fact, similar to the search of the hyper-parameter alpha for Lasso/Ridge regression so we should perform it on the training set using the test set for the final evaluation of the feature subset.

The subset of features selected using the k-nearest neighbors (kNN) algorithm may be sparser compared to the subset selected using linear regression for a few reasons:

1. **Different optimization criteria**: The optimization criteria used by the kNN and linear regression algorithms may be different, leading to different feature subsets being selected. For example, the kNN algorithm may prioritize selecting features that are more discriminative for the classification or regression task, while linear regression may prioritize selecting features that are more strongly correlated with the target variable.

2. **Different feature importance measures**: The kNN and linear regression algorithms may use different measures of feature importance, which could affect the sparsity of the selected feature subset. For example, the kNN algorithm may use a measure such as mutual information or chi-squared to identify important features, while linear regression may use a measure such as the coefficient of determination or the t-statistic.

3. **Different assumptions about the data**: The kNN and linear regression algorithms make different assumptions about the structure of the data, which could also affect the sparsity of the selected feature subset. The kNN algorithm assumes that points that are similar in feature space are likely to belong to the same class or have similar target values, while linear regression assumes that the relationship between the features and the target is linear. These different assumptions could lead to different feature subsets being selected.

Overall, the sparsity of the selected feature subset will depend on the specific characteristics of the data and the optimization criteria being used. Different algorithms and feature selection methods may yield different subsets of features, and it may be necessary to experiment with different approaches to find the one that works best for your data and problem.